# One-vs-All Kernel Perceptron with Gaussian Kernel

In [1]:
import numpy as np
from tqdm import tqdm
from tabulate import tabulate

In [8]:
class MultiClassPerceptronOVA:
    def __init__(self, train_data, train_labels, test_data, test_labels, kernel_type, kernel_param, epochs):
        """
        Initialize the MultiClassPerceptronOVA class.

        Inputs:
        - train_data (ndarray): Training data features.
        - train_labels (ndarray): Labels corresponding to training data.
        - test_data (ndarray): Test data features.
        - test_labels (ndarray): Labels corresponding to test data.
        - kernel_type (str): Type of kernel ('p' for polynomial, 'g' for Gaussian).
        - kernel_param (float): Parameter for the kernel (degree for polynomial, sigma for Gaussian).
        - epochs (int): Number of training epochs.

        Output:
        None
        """
        self.train_data = train_data
        self.train_labels = train_labels
        self.test_data = test_data
        self.test_labels = test_labels
        self.epochs = epochs
        self.classes = np.unique(np.hstack((train_labels, test_labels)))
        self.num_classes = len(self.classes)
        
        self.train_count = train_labels.size
        self.test_count = test_labels.size
        self.kernel_type = kernel_type
        self.kernel_param = kernel_param
        self.kernel_train = self.kernel_compute(train_data, train_data, kernel_type, kernel_param)
        self.kernel_test = self.kernel_compute(test_data, train_data, kernel_type, kernel_param)
        self.alpha = np.zeros((self.num_classes, self.train_count))

    def kernel_compute(self, A, B, k_type, k_arg):
        """
        Compute the kernel matrix.

        Inputs:
        - A (ndarray): First data matrix.
        - B (ndarray): Second data matrix.
        - k_type (str): Type of kernel ('p' for polynomial, 'g' for Gaussian).
        - k_arg (float): Parameter for the kernel (degree for polynomial, sigma for Gaussian).

        Output:
        - ndarray: Computed kernel matrix.
        """
        if k_type == 'p':
            return (A @ B.T) ** k_arg
        if k_type == 'g':
            return self.gaussian(A, B, k_arg)

    def gaussian(self, X, Y, c):
        """
        Compute the Gaussian kernel (vectorized).

        Inputs:
        - X (ndarray): First data matrix.
        - Y (ndarray): Second data matrix.
        - c (float): Gaussian kernel parameter.

        Output:
        - ndarray: Computed Gaussian kernel matrix.
        """
        pairwise_sq_dists = np.sum(X**2, axis=1).reshape(-1, 1) + np.sum(Y**2, axis=1) - 2 * np.dot(X, Y.T)
        return np.exp(-c * pairwise_sq_dists)    
    
    def train(self):
        """
        Train the OvA Kernel Perceptron.

        Output:
        - float: Training error rate after the final epoch.
        """
        for _ in range(self.epochs):
            misclassifications = 0
            for idx in range(self.train_count):
                for label_idx, label in enumerate(self.classes):
                    binary_label = 1 if self.train_labels[idx] == label else -1
                    prediction = np.sum(self.alpha[label_idx] * self.kernel_train[:, idx])
                    if np.sign(prediction) != binary_label:
                        self.alpha[label_idx, idx] += binary_label
                        misclassifications += 1
            print(f"Epoch complete. Misclassifications: {misclassifications}")
        return np.mean(self.predict(self.train_data, True) != self.train_labels)

    def predict(self, data, training_phase):
        """
        Predict class labels for given data.

        Inputs:
        - data (ndarray): Data points to classify.
        - training_phase (bool): Whether the prediction is for training (True) or testing (False).

        Output:
        - ndarray: Predicted class labels.
        """
        kernel_matrix = self.kernel_train if training_phase else self.kernel_test
        class_scores = np.zeros((data.shape[0], self.num_classes))
        for class_idx in range(self.num_classes):
            class_scores[:, class_idx] = self.alpha[class_idx] @ kernel_matrix.T
        return self.classes[np.argmax(class_scores, axis=1)]

    def test(self):
        """
        Evaluate the OvA Kernel Perceptron on the test set.

        Output:
        - float: Test error rate.
        """
        return np.mean(self.predict(self.test_data, False) != self.test_labels)


(a) - Explain how to choose the set S and values chosen for S (S is a set of values for parameter c). Run some initial experiments to decide.

In [3]:
# Split data into training and testing sets
def train_test_split(X, y, train_ratio=0.8, shuffle=True, random_seed=None):
    """
    Split dataset into training and testing sets.

    Inputs:
    - X (ndarray): Feature data.
    - y (ndarray): Corresponding labels.
    - train_ratio (float): Proportion of data for training (default is 0.8).
    - shuffle (bool): Whether to shuffle data before splitting (default is True).
    - random_seed (int or None): Random seed for reproducibility (default is None).

    Outputs:
    - Tuple[ndarray, ndarray, ndarray, ndarray]: 
      Training features, test features, training labels, test labels.
    """
    if shuffle:
        np.random.seed(random_seed)
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        X, y = X[indices], y[indices]
    n_train = int(train_ratio * len(X))
    return X[:n_train], X[n_train:], y[:n_train], y[n_train:]

In [ ]:
# Load dataset
X, y = np.loadtxt("zipcombo.dat")[:, 1:], np.loadtxt("zipcombo.dat")[:, 0].astype(int)

In [4]:
# Initial range of Gaussian kernel parameters
initial_c_values = [0.001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.5]  # Candidate values for Gaussian kernel parameter

# Set random seed for reproducibility and initialize list for test errors
np.random.seed(42)
test_errors = []

def test_c(train_X, test_X, train_y, test_y, c):
    """
    Train and test the model for a specific Gaussian kernel parameter.

    Inputs:
    - train_X, test_X: Training and testing features.
    - train_y, test_y: Training and testing labels.
    - c: Gaussian kernel parameter (controls the kernel width).

    Output:
    - Test error rate for the given kernel parameter.
    """
    # Initialize and train the model with the Gaussian kernel
    model = MultiClassPerceptronOVA(train_X, train_y, test_X, test_y, kernel_type='g', kernel_param=c, epochs=5)
    model.train()
    return model.test()  # Return test error rate

print("Evaluating Gaussian kernel parameters...")
# Loop through initial c values to evaluate model performance
for c in tqdm(initial_c_values, desc="Evaluating c values"):
    # Split the dataset into training and testing sets (80%-20% split)
    train_X, test_X, train_y, test_y = train_test_split(X, y, train_ratio=0.8, shuffle=True)
    error_rate = test_c(train_X, test_X, train_y, test_y, c)  # Test model with the given c
    test_errors.append(error_rate)  # Store the test error for the current c
    print(f"c = {c:.4f}, Test Error = {error_rate:.4f}")

# Analyze and print the test errors for all c values
print("\nAnalysis of initial c evaluations:")
for c, error in zip(initial_c_values, test_errors):
    print(f"c = {c:.4f}: Test Error = {error:.4f}")



Evaluating Gaussian kernel parameters...


Evaluating c values:   0%|          | 0/7 [00:00<?, ?it/s]

Epoch complete. Misclassifications: 3085
Epoch complete. Misclassifications: 1979
Epoch complete. Misclassifications: 1726
Epoch complete. Misclassifications: 1506
Epoch complete. Misclassifications: 1352


Evaluating c values:  14%|█▍        | 1/7 [00:32<03:14, 32.42s/it]

c = 0.0010, Test Error = 0.0774
Epoch complete. Misclassifications: 1806
Epoch complete. Misclassifications: 701
Epoch complete. Misclassifications: 462
Epoch complete. Misclassifications: 322
Epoch complete. Misclassifications: 217


...

c = 0.0500, Test Error = 0.0376
Epoch complete. Misclassifications: 2062
Epoch complete. Misclassifications: 480
Epoch complete. Misclassifications: 83
Epoch complete. Misclassifications: 10
Epoch complete. Misclassifications: 5


Evaluating c values:  86%|████████▌ | 6/7 [03:12<00:31, 31.68s/it]

c = 0.1000, Test Error = 0.0435
Epoch complete. Misclassifications: 2508
Epoch complete. Misclassifications: 615
Epoch complete. Misclassifications: 69
Epoch complete. Misclassifications: 14
Epoch complete. Misclassifications: 0


Evaluating c values: 100%|██████████| 7/7 [03:41<00:00, 31.67s/it]

c = 0.5000, Test Error = 0.0554

Analysis of initial c evaluations:
c = 0.0010: Test Error = 0.0774
c = 0.0050: Test Error = 0.0285
c = 0.0100: Test Error = 0.0274
c = 0.0200: Test Error = 0.0258
c = 0.0500: Test Error = 0.0376
c = 0.1000: Test Error = 0.0435
c = 0.5000: Test Error = 0.0554


(b): Perform Q3 procedure with the Gaussian Kernel.

In [6]:
# Initial range of Gaussian kernel parameters
c_list = [0.001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.5]  # Gaussian kernel parameter candidates

# Set random seed for reproducibility and initialize lists to store results
np.random.seed(42)
train_error, test_error, train_std, test_std = [], [], [], []

# Evaluate training and test errors for each Gaussian kernel parameter
for c in c_list:
    single_run_train_errors, single_run_test_errors = [], []  # Temporary lists for errors in 20 runs
    print(f"Running for parameter = {c}...")

    # Perform 20 runs to compute error statistics for the current c
    for run in tqdm(range(20), desc=f"Parameter {c}"):
        # Split the dataset into training and testing sets with a consistent random seed
        train_X, test_X, train_y, test_y = train_test_split(X, y, train_ratio=0.8, random_seed=run)
        
        # Initialize and train the model with the Gaussian kernel and current parameter
        model = MultiClassPerceptronOVA(train_X, train_y, test_X, test_y, kernel_type='g', kernel_param=c, epochs=5)
        model.train()

        # Predict outcomes on training and testing sets
        train_predictions = model.predict(train_X, training_phase=True)
        test_predictions = model.predict(test_X, training_phase=False)

        # Compute error rates for training and testing sets
        train_e = np.mean(train_predictions != train_y)
        test_e = np.mean(test_predictions != test_y)

        # Store errors for this run
        single_run_train_errors.append(train_e)
        single_run_test_errors.append(test_e)

    # Compute mean and standard deviation of errors across 20 runs for the current c
    train_error.append(np.mean(single_run_train_errors))
    train_std.append(np.std(single_run_train_errors))
    test_error.append(np.mean(single_run_test_errors))
    test_std.append(np.std(single_run_test_errors))

    # Print the results for the current c
    print(f"c = {c}: Train Error = {train_error[-1]:.4f}, Test Error = {test_error[-1]:.4f}")



Running for parameter = 0.001...


Parameter 0.001:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch complete. Misclassifications: 3021
Epoch complete. Misclassifications: 1865
Epoch complete. Misclassifications: 1625
Epoch complete. Misclassifications: 1481
Epoch complete. Misclassifications: 1367


Parameter 0.001:   5%|▌         | 1/20 [00:23<07:32, 23.82s/it]

Epoch complete. Misclassifications: 3102
Epoch complete. Misclassifications: 1894
Epoch complete. Misclassifications: 1661
Epoch complete. Misclassifications: 1501
Epoch complete. Misclassifications: 1300


...

Epoch complete. Misclassifications: 2476
Epoch complete. Misclassifications: 578
Epoch complete. Misclassifications: 83
Epoch complete. Misclassifications: 10
Epoch complete. Misclassifications: 0


Parameter 0.5:  95%|█████████▌| 19/20 [10:08<00:33, 33.06s/it]

Epoch complete. Misclassifications: 2314
Epoch complete. Misclassifications: 592
Epoch complete. Misclassifications: 82
Epoch complete. Misclassifications: 4
Epoch complete. Misclassifications: 0


Parameter 0.5: 100%|██████████| 20/20 [10:40<00:00, 32.03s/it]

c = 0.5: Train Error = 0.0000, Test Error = 0.0676


In [7]:
# Tabulate results
data = []
for i in range(len(c_list)):
    train_result = f"{train_error[i] * 100:.4f}±{train_std[i]:.4f}"
    test_result = f"{test_error[i] * 100:.4f}±{test_std[i]:.4f}"
    data.append([c_list[i], train_result, test_result])

print(tabulate(
    data,
    headers=["c", "Mean Train Error Rates (%)", "Mean Test Error Rates (%)"],
    tablefmt="simple_outline",
    stralign="center"
))

┌───────┬──────────────────────────────┬─────────────────────────────┐
│     c │  Mean Train Error Rates (%)  │  Mean Test Error Rates (%)  │
├───────┼──────────────────────────────┼─────────────────────────────┤
│ 0.001 │        6.8621±0.0294         │        8.7876±0.0283        │
│ 0.005 │        0.3865±0.0014         │        3.3226±0.0051        │
│ 0.01  │        0.0746±0.0005         │        2.8199±0.0046        │
│ 0.02  │        0.0229±0.0001         │        2.5860±0.0043        │
│ 0.05  │        0.0430±0.0009         │        3.9005±0.0051        │
│ 0.1   │        0.0161±0.0003         │        5.3602±0.0046        │
│ 0.5   │        0.0000±0.0000         │        6.7581±0.0063        │
└───────┴──────────────────────────────┴─────────────────────────────┘


(c): Perform Q4 procedure with the Gaussian Kernel. Cross Validation with 20 Runs.

In [18]:
# Cross-validation function
def cross_validation_splits(features, labels, folds=5):
    """
    Generate k-fold cross-validation splits.

    Inputs:
    - features (ndarray): Feature data.
    - labels (ndarray): Corresponding labels.
    - folds (int): Number of cross-validation folds (default is 5).

    Output:
    - List[Tuple[ndarray, ndarray, ndarray, ndarray]]:
      A list of tuples, where each tuple contains (train_features, valid_features, train_labels, valid_labels).
    """
    split_size = len(features) // folds  # Calculate the size of each fold
    results = []

    # Create splits for cross-validation
    for fold in range(folds):
        start, end = fold * split_size, (fold + 1) * split_size
        valid_features = features[start:end]  # Validation features for the current fold
        valid_labels = labels[start:end]  # Validation labels for the current fold
        # Combine remaining data into training set
        train_features = np.concatenate([features[:start], features[end:]]) if start > 0 else features[end:]
        train_labels = np.concatenate([labels[:start], labels[end:]]) if start > 0 else labels[end:]
        results.append((train_features, valid_features, train_labels, valid_labels))

    return results  # Return all training-validation splits


In [19]:
# Cross-validation setup
np.random.seed(42)  # Set seed for reproducibility
c_list = [0.001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.5]  # Candidate values for Gaussian kernel parameter
c_stars, test_errors, train_errors = [], [], []  # Store optimal c values and errors across iterations

# Perform 20 cross-validation runs to find the best kernel parameter c
for iteration in tqdm(range(20), desc="Cross-Validation Runs"):
    # Split data into training and testing sets for this iteration
    train_X, test_X, train_y, test_y = train_test_split(X, y, train_ratio=0.8, random_seed=iteration)
    optimal_c, lowest_error = 0, float('inf')  # Initialize optimal c and lowest validation error

    # Evaluate each candidate c using 5-fold cross-validation
    for c in c_list:
        print(f"Iteration {iteration + 1}: Evaluating c = {c}...")
        validation_errors = []  # List to store validation errors for this c

        # Generate k-fold splits and evaluate the model on each fold
        folds = cross_validation_splits(train_X, train_y, folds=5)
        for fold_idx, (fold_train_X, fold_valid_X, fold_train_y, fold_valid_y) in enumerate(folds):
            print(f"Iteration {iteration + 1}, c = {c}, Fold {fold_idx + 1}/5...")
            model = MultiClassPerceptronOVA(
                fold_train_X, fold_train_y, fold_valid_X, fold_valid_y,
                kernel_type='g', kernel_param=c, epochs=5
            )
            model.train()  # Train the model on the fold's training set
            validation_errors.append(model.test())  # Test on the fold's validation set

        # Calculate the average validation error for the current c
        avg_validation_error = np.mean(validation_errors)
        print(f"Iteration {iteration + 1}, c = {c}: Mean Validation Error = {avg_validation_error:.4f}")

        # Update the optimal c if the current c gives a lower validation error
        if avg_validation_error < lowest_error:
            lowest_error = avg_validation_error
            optimal_c = c

    print(f"Iteration {iteration + 1}: Best c* = {optimal_c}, Validation Error = {lowest_error:.4f}")

    # Train and test the final model with the optimal c for this iteration
    final_model = MultiClassPerceptronOVA(
        train_X, train_y, test_X, test_y,
        kernel_type='g', kernel_param=optimal_c, epochs=5
    )
    train_errors.append(final_model.train())  # Store training error
    test_errors.append(final_model.test())  # Store test error
    c_stars.append(optimal_c)  # Store the optimal c for this iteration

Cross-Validation Runs:   0%|          | 0/20 [00:00<?, ?it/s]

Run 1: Evaluating c = 0.001...
Run 1, c = 0.001, Fold 1/5...
Epoch 1/5, Training Errors: 2581
Epoch 2/5, Training Errors: 1570
Epoch 3/5, Training Errors: 1334
Epoch 4/5, Training Errors: 1245
Epoch 5/5, Training Errors: 1056
Run 1, c = 0.001, Fold 2/5...
Epoch 1/5, Training Errors: 2614
Epoch 2/5, Training Errors: 1585
Epoch 3/5, Training Errors: 1312
Epoch 4/5, Training Errors: 1228
Epoch 5/5, Training Errors: 1125
Run 1, c = 0.001, Fold 3/5...
Epoch 1/5, Training Errors: 2623
Epoch 2/5, Training Errors: 1628
Epoch 3/5, Training Errors: 1361
Epoch 4/5, Training Errors: 1259
Epoch 5/5, Training Errors: 1141
Run 1, c = 0.001, Fold 4/5...
Epoch 1/5, Training Errors: 2606
Epoch 2/5, Training Errors: 1614
Epoch 3/5, Training Errors: 1403
Epoch 4/5, Training Errors: 1215
Epoch 5/5, Training Errors: 1061
Run 1, c = 0.001, Fold 5/5...
Epoch 1/5, Training Errors: 2568
Epoch 2/5, Training Errors: 1578
Epoch 3/5, Training Errors: 1330
Epoch 4/5, Training Errors: 1202
Epoch 5/5, Training Errors:

Cross-Validation Runs:   5%|▌         | 1/20 [11:01<3:29:27, 661.44s/it]

c*:  0.02 train error rate: 0.00026888948642108095 , test error rate:  0.02849462365591398
Run 2: Evaluating c = 0.001...
Run 2, c = 0.001, Fold 1/5...
Epoch 1/5, Training Errors: 2623
Epoch 2/5, Training Errors: 1651
Epoch 3/5, Training Errors: 1419
Epoch 4/5, Training Errors: 1249
Epoch 5/5, Training Errors: 1177
Run 2, c = 0.001, Fold 2/5...
Epoch 1/5, Training Errors: 2583
Epoch 2/5, Training Errors: 1592
Epoch 3/5, Training Errors: 1425
Epoch 4/5, Training Errors: 1275
Epoch 5/5, Training Errors: 1098
Run 2, c = 0.001, Fold 3/5...
Epoch 1/5, Training Errors: 2589
Epoch 2/5, Training Errors: 1668
Epoch 3/5, Training Errors: 1357
Epoch 4/5, Training Errors: 1239
Epoch 5/5, Training Errors: 1183
Run 2, c = 0.001, Fold 4/5...
Epoch 1/5, Training Errors: 2588
Epoch 2/5, Training Errors: 1615
Epoch 3/5, Training Errors: 1402
Epoch 4/5, Training Errors: 1222
Epoch 5/5, Training Errors: 1070
Run 2, c = 0.001, Fold 5/5...
Epoch 1/5, Training Errors: 2653
Epoch 2/5, Training Errors: 1625
Ep

Cross-Validation Runs:  10%|█         | 2/20 [22:08<3:19:26, 664.78s/it]

...

c*:  0.02 train error rate: 0.00026888948642108095 , test error rate:  0.027419354838709678
Run 19: Evaluating c = 0.001...
Run 19, c = 0.001, Fold 1/5...
Epoch 1/5, Training Errors: 2618
Epoch 2/5, Training Errors: 1651
Epoch 3/5, Training Errors: 1402
Epoch 4/5, Training Errors: 1257
Epoch 5/5, Training Errors: 1117
Run 19, c = 0.001, Fold 2/5...
Epoch 1/5, Training Errors: 2630
Epoch 2/5, Training Errors: 1694
Epoch 3/5, Training Errors: 1400
Epoch 4/5, Training Errors: 1264
Epoch 5/5, Training Errors: 1143
Run 19, c = 0.001, Fold 3/5...
Epoch 1/5, Training Errors: 2642
Epoch 2/5, Training Errors: 1670
Epoch 3/5, Training Errors: 1393
Epoch 4/5, Training Errors: 1225
Epoch 5/5, Training Errors: 1096
Run 19, c = 0.001, Fold 4/5...
Epoch 1/5, Training Errors: 2595
Epoch 2/5, Training Errors: 1676
Epoch 3/5, Training Errors: 1428
Epoch 4/5, Training Errors: 1219
Epoch 5/5, Training Errors: 1093
Run 19, c = 0.001, Fold 5/5...
Epoch 1/5, Training Errors: 2585
Epoch 2/5, Training Errors: 

Cross-Validation Runs:  95%|█████████▌| 19/20 [3:18:48<09:31, 571.13s/it]

c*:  0.02 train error rate: 0.00040333422963162143 , test error rate:  0.027419354838709678
Run 20: Evaluating c = 0.001...
Run 20, c = 0.001, Fold 1/5...
Epoch 1/5, Training Errors: 2660
Epoch 2/5, Training Errors: 1671
Epoch 3/5, Training Errors: 1372
Epoch 4/5, Training Errors: 1233
Epoch 5/5, Training Errors: 1124
Run 20, c = 0.001, Fold 2/5...
Epoch 1/5, Training Errors: 2639
Epoch 2/5, Training Errors: 1655
Epoch 3/5, Training Errors: 1365
Epoch 4/5, Training Errors: 1289
Epoch 5/5, Training Errors: 1162
Run 20, c = 0.001, Fold 3/5...
Epoch 1/5, Training Errors: 2668
Epoch 2/5, Training Errors: 1605
Epoch 3/5, Training Errors: 1311
Epoch 4/5, Training Errors: 1282
Epoch 5/5, Training Errors: 1098
Run 20, c = 0.001, Fold 4/5...
Epoch 1/5, Training Errors: 2574
Epoch 2/5, Training Errors: 1616
Epoch 3/5, Training Errors: 1292
Epoch 4/5, Training Errors: 1232
Epoch 5/5, Training Errors: 1065
Run 20, c = 0.001, Fold 5/5...
Epoch 1/5, Training Errors: 2582
Epoch 2/5, Training Errors: 

Cross-Validation Runs: 100%|██████████| 20/20 [3:28:02<00:00, 624.15s/it]

c*:  0.02 train error rate: 0.00013444474321054048 , test error rate:  0.03118279569892473


In [20]:
# Tabulate results
data_table = []
for run_idx in range(20):
    train_result = f"{train_errors[run_idx] * 100:.4f}"
    test_result = f"{test_errors[run_idx] * 100:.4f}"
    data_table.append([run_idx + 1, c_stars[run_idx], train_result, test_result])

print(tabulate(
    data_table,
    headers=["Run", "Best c*", "Train Error Rate (%)", "Test Error Rate (%)"],
    tablefmt="simple_outline",
    numalign="center"
))

┌───────┬──────┬─────────────────────────┬────────────────────────┐
│  Run  │  c*  │  Train Error Rates (%)  │  Test Error Rates (%)  │
├───────┼──────┼─────────────────────────┼────────────────────────┤
│   1   │ 0.02 │         0.0269          │         2.8495         │
│   2   │ 0.02 │         0.0538          │         2.3656         │
│   3   │ 0.02 │         0.0269          │         1.4516         │
│   4   │ 0.02 │         0.0403          │         3.2258         │
│   5   │ 0.01 │         0.0807          │         3.0108         │
│   6   │ 0.02 │         0.0269          │         3.172          │
│   7   │ 0.02 │         0.0134          │         3.0108         │
│   8   │ 0.02 │         0.0134          │         2.4194         │
│   9   │ 0.02 │         0.0134          │         2.2043         │
│  10   │ 0.01 │          0.121          │         3.172          │
│  11   │ 0.02 │         0.0269          │         2.0968         │
│  12   │ 0.02 │         0.0134          │      

In [22]:
# Summary
print("\nCross-Validation Summary:")
print(f"Mean c*: {np.mean(c_stars):.2f} ± {np.std(c_stars):.3f}")
print(f"Mean Train Error Rate: {np.mean(train_errors) * 100:.4f} ± {np.std(train_errors):.4f}")
print(f"Mean Test Error Rate: {np.mean(test_errors) * 100:.4f} ± {np.std(test_errors):.4f}")


Cross-Validation Summary:
Mean c*: 0.02 ± 0.004
Mean Train Error Rate: 0.0323 ± 0.0003
Mean Test Error Rate: 2.6828 ± 0.0045
